In [6]:
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("d:\\data\\ISTFinal\\FinalProjectDataSentiment.csv")
data = data.replace(np.nan, '', regex=True)

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)

trainY = train['GoodComment']
trainX = train[['MinutesAfterStory','SameAuthor','compound','neg','neu','pos']]
testY = test['GoodComment']
testX = test[['MinutesAfterStory','SameAuthor','compound','neg','neu','pos']]

In [10]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=10,random_state=0).fit(trainX, trainY)

print(clf.feature_importances_)
rfPredict = clf.predict_proba(testX)

print(rfPredict)

[0.30372646 0.01741712 0.14240723 0.20522418 0.17709411 0.15413091]
[[0.50562795 0.49437205]
 [0.38371835 0.61628165]
 [0.57749127 0.42250873]
 ...
 [0.58075509 0.41924491]
 [0.61487712 0.38512288]
 [0.53431159 0.46568841]]


In [12]:
from sklearn.naive_bayes import MultinomialNB

gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), stop_words='english')
X_train_vec = gram12_count_vectorizer.fit_transform(train.CommentText)
X_test_vec = gram12_count_vectorizer.transform(test.CommentText)

nb_clf= MultinomialNB()
nb_clf.fit(X_train_vec,train.GoodComment)

nbPredict = nb_clf.predict_proba(X_test_vec)

print(nbPredict)

[[7.19988550e-01 2.80011450e-01]
 [5.70161867e-01 4.29838133e-01]
 [9.99926161e-01 7.38388701e-05]
 ...
 [1.00000000e+00 5.94262714e-21]
 [9.26236680e-01 7.37633195e-02]
 [9.93257013e-01 6.74298661e-03]]


In [38]:
from sklearn.metrics import accuracy_score

rfDf = pd.DataFrame(rfPredict)
nbDf = pd.DataFrame(nbPredict)

combinedDf = pd.DataFrame(list(zip(rfDf[0], nbDf[0])))
combinedPredictions = combinedDf.mean(1)

pred = combinedPredictions < 0.5
accuracy_score(testY, pred)

0.663570998019461